IMPORTS

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA, TruncatedSVD, IncrementalPCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures,MinMaxScaler,RobustScaler, LabelEncoder
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.feature_selection import SelectKBest, f_regression, VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer,KNNImputer
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns



Read the files 

In [2]:
train_df = pd.read_csv("/Users/talalkhan/Documents/Data Sets/Second Challange/train.csv")
test_df = pd.read_csv("/Users/talalkhan/Documents/Data Sets/Second Challange/test.csv")



Drop sub_area 

In [3]:
'''# Identify columns that start with 'sub_area_'
sub_area_columns = [col for col in train_df.columns if col.startswith('sub_area_')]

# Drop these columns
train_df.drop(columns=sub_area_columns, inplace=True)

train_df = train_df.drop(columns=['sub_area'])
test_df = test_df.drop(columns=['sub_area','row ID'])
'''
# Drop columns starting with 'sub_area_'
sub_area_columns = [col for col in train_df.columns if col.startswith('sub_area_')]
train_df.drop(columns=sub_area_columns, inplace=True)

# Separate features and target
X_train = train_df.drop('price_doc', axis=1)
y_train = train_df['price_doc']
X_test = test_df.drop(columns=sub_area_columns)

# Split the data into training and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Handle Categorical Data

In [ ]:
'''#do onehot encoding for categorical columns
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)
'''


In [4]:
# Identifying categorical and numerical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
numerical_cols = X_train.select_dtypes(exclude=['object', 'category']).columns

# Creating preprocessors for numerical and categorical data
numerical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='mean')),
    ('imputer', KNNImputer(n_neighbors=500)),  # KNN imputation
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    #('label', LabelEncoder())])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [58]:
train_df.head()


,full_sq,life_sq,floor,product_type,sub_area,area_m,raion_popul,green_zone_part,indust_part,children_preschool,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,43.0,27.0,4.0,Investment,Bibirevo,6407578.100,155572.0,0.189727,0.000070,9576.0,...,9.0,4.0,0.0,13.0,22.0,1.0,0.0,52.0,4.0,5850000.0
1,34.0,19.0,3.0,Investment,Nagatinskij Zaton,9589336.912,115352.0,0.372602,0.049637,6880.0,...,15.0,3.0,0.0,15.0,29.0,1.0,10.0,66.0,14.0,6000000.0
2,43.0,29.0,2.0,Investment,Tekstil'shhiki,4808269.831,101708.0,0.112560,0.118537,5879.0,...,10.0,3.0,0.0,11.0,27.0,0.0,4.0,67.0,10.0,5700000.0
3,77.0,77.0,4.0,Investment,Basmannoe,8398460.622,108171.0,0.015234,0.037316,5706.0,...,319.0,108.0,17.0,135.0,236.0,2.0,91.0,195.0,14.0,16331452.0
4,67.0,46.0,14.0,Investment,Nizhegorodskoe,7506452.020,43795.0,0.007670,0.486246,2418.0,...,62.0,14.0,1.0,53.0,78.0,1.0,20.0,113.0,17.0,9100000.0


Correlation Finding

In [ ]:
#'''
# Select only numeric columns for correlation
numeric_cols = train_df.select_dtypes(include=[np.number])

# Calculate correlation with price_doc
correlation = numeric_cols.corr()['price_doc'].sort_values(ascending=False)
#'''
'''
# Assume 'Investment' is a categorical column
one_hot_encoded_data = pd.get_dummies(train_df)

# Now calculate the correlation
correlation = one_hot_encoded_data.corr()['price_doc'].sort_values(ascending=False)
'''
print(correlation)


In [ ]:
# Set a threshold for correlation
correlation_threshold = 0.05

# Find columns with correlation less than the threshold
columns_to_drop = correlation[abs(correlation) < correlation_threshold].index

# Drop these columns from the dataset
train_df.drop(columns=columns_to_drop, inplace=True)
# Drop the same columns from the test dataset
test_df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
# Check the remaining columns
print(train_df.columns)
print(test_df.columns)

P-Value (OLS Reg)

In [ ]:
# Add a constant to the model (intercept)
X_train = sm.add_constant(X_train)

# Fit the OLS model
model = sm.OLS(y_train, X_train).fit()(n_jobs=-1)

# Summary of the model
print(model.summary())


In [1]:
# Set a threshold for p-values, e.g., 0.05
p_value_threshold = 0.05

# Filter features
significant_features = [feature for feature in X_train.columns if model.pvalues[feature] < p_value_threshold]
#significant_features2 = [feature for feature in X_test.columns if model.pvalues[feature] < p_value_threshold]

# Select only significant features for your final model
X_filtered = X_train[significant_features]
X_test_processed = X_test[significant_features] 

NameError: name 'X_train' is not defined

Feature Selection

In [5]:
# Apply feature selection
feature_selection = SelectKBest(score_func=f_regression, k=80)  # Adjust 'k' as needed


In [ ]:
# Initialize VarianceThreshold
feature_selection = VarianceThreshold(threshold=0.05)  # Adjust the threshold as needed

PCA

In [6]:
# Applying PCA
pca = PCA(n_components=0.95)  # Retain 95% of variance
#svd = TruncatedSVD(n_components=100)  # You may need to tune the number of components

# Applying Incremental PCA
n_batches = 100  # Adjust the number of batches
ipca = IncrementalPCA(n_components=100)  # Adjust the number of components


In [ ]:
# Forward Feature Selection
sfs = SFS(LinearRegression(), 
          k_features='best', 
          forward=True, 
          scoring='neg_mean_squared_error',
          cv=5,
          n_jobs=-1)


MODELS

In [7]:
# Train the model using selected features
model = LinearRegression()
#model.fit(X_train.iloc[:, list(selected_features)], y_train)


In [8]:
# Generate polynomial features
polynomial_transformer = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

Model the final Pipeline

In [9]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('feature_selection', feature_selection),
                            ('polynomial', polynomial_transformer),
                            #('svd', svd),
                            #('pca', pca),
                            ('ipca', ipca),
                           ('linear_model', model)
                           ])

pipeline.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=500)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  Index(['full_sq', 'life_sq', 'floor', 'area_m', 'raion_popul',
       'green_zone_part', 'indust_part', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       ...
       'cafe_count_5000_price_1500...
       'big_market_raion', 'nuclear_reactor_raion', 'detention_facility_raion',
       'water_1line', 'big_road1_1line', 'railroad_1line', 'ecology'],
      dtype='object'))])),
                ('feature_selection',
                 SelectKBest(k=80,
                             score_func=<function f_regression at 0x17ea3ed40>)),
                ('polynomial', PolynomialFeatures(include_bias=False)),
                ('ipca', IncrementalPCA(n_components=100)),
                ('linear_model', LinearRegression())])

In [ ]:
'''
# Predict on validation set
y_pred = pipeline.predict(X_val)

# Calculate MSE
mse = mean_squared_error(y_val, y_pred)
print(f"Mean Squared Error: {mse}")
'''

In [ ]:
model.fit(X_train, y_train)


In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(X_test, y_pred)
print(f'Mean Squared Error: {mse}')



In [ ]:
# Predict using the model
predicted_prices = model.predict(X_test)

print(predicted_prices)

Pipeline Predict

In [10]:
predicted_prices = pipeline.predict(X_test)


Make the CSV file

In [53]:
#Saving predictions to a CSV file for Kaggle submission

result_df = pd.DataFrame({'row ID': test_df['row ID'], 'price_doc': predicted_prices})
result_df.to_csv('submission16_25253.csv', index=False)
